In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [2]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [3]:
np.random.seed(1234) ### DOES NOT WORK???
w_length = 100
n_countries = 2

In [6]:
mx = pd.DataFrame()
us = pd.DataFrame()
for file in glob.glob("Raw Track Data\\*.csv"):
    name = file[15:-4]
    new = pd.read_csv(file)
    if name[:2] == "AD":
        mx = mx.append(new)
    if name[:2] == "DZ":
        us = us.append(new)

In [7]:
mx = mx.drop(["confidence", "loudness_start", "loudness_max_time"], axis = 1)
us = us.drop(["confidence", "loudness_start", "loudness_max_time"], axis = 1)

In [8]:
print(mx.shape)
mx = mx.drop_duplicates(["track_id", "start"])
print(mx.shape)

(140769, 28)
(140769, 28)


In [9]:
print(us.shape)
us = us.drop_duplicates(["track_id", "start"])
print(us.shape)

(259215, 28)
(224323, 28)


In [10]:
print("MX", mx.shape)
print("US", us.shape)

MX (140769, 28)
US (224323, 28)


### Check if the playlists have common songs

In [11]:
sum(us.track_id.isin(mx.track_id))

0

In [12]:
dupes = us["track_id"].loc[us.track_id.isin(mx.track_id)]
mx = mx.loc[~(mx["track_id"].isin(dupes))]
us = us.loc[~(us["track_id"].isin(dupes))]
print("MX", mx.shape)
print("US", us.shape)

MX (140769, 28)
US (224323, 28)


### Input array for US

In [13]:
new_pos = list(us.track_id.index[us.track_id.shift(1) != us.track_id]) # indices where the song changes
new_pos.append(max(us.track_id.index) + 1) # add a new index to know where the last song ends
split_pos = []
for i in range(len(new_pos)-1):
    split_pos = split_pos + list(range(new_pos[i], new_pos[i+1], w_length))
split_pos = split_pos[1:]
us_train = np.split(us.iloc[:,:28].to_numpy(), split_pos)
# drop the short sequences
short_seqs = []
temp = [] 
for i, value in enumerate(us_train):
    if value.shape[0] == w_length:
        temp.append(value)
us_train = temp
us_train = np.stack(us_train)
val_index = np.isin(us_train[:,:,27],
                    np.random.choice(us.track_id.unique(), np.int(len(us.track_id.unique())/10)))
val_index = val_index.sum(1) != 0
us_val = us_train[val_index,:,:26] # drop track id
us_train = us_train[np.logical_not(val_index),:,:26] # drop track id
us_train = us_train.astype("float64")
us_val = us_val.astype("float64")

If we want songs to have the same number of observations, we can determine which element in the list returned by np.slit corresponds to which song (using new_pos) and randomly overpopulate accordingly. An alternative is to do this with an online batching if we have memory problems.

### Input array for MX

In [14]:
new_pos = list(mx.track_id.index[mx.track_id.shift(1) != mx.track_id]) # indices where the song changes
new_pos.append(max(mx.track_id.index) + 1) # add a new index to know where the last song ends
split_pos = []
for i in range(len(new_pos)-1):
    split_pos = split_pos + list(range(new_pos[i], new_pos[i+1], w_length))
split_pos = split_pos[1:]
mx_train = np.split(mx.iloc[:,:28].to_numpy(), split_pos)
# drop the short sequences
short_seqs = []
temp = [] 
for i, value in enumerate(mx_train):
    if value.shape[0] == w_length:
        temp.append(value)
mx_train = temp
mx_train = np.stack(mx_train)
val_index = np.isin(mx_train[:,:,27],
                    np.random.choice(mx.track_id.unique(), np.int(len(mx.track_id.unique())/10)))
val_index = val_index.sum(1) != 0
mx_val = mx_train[val_index,:,:26] # drop track id
mx_train = mx_train[np.logical_not(val_index),:,:26] # drop track id
mx_train = mx_train.astype("float64")
mx_val = mx_val.astype("float64")

### Input and Output arrays

In [15]:
train_input = np.concatenate([us_train, mx_train])
val_input = np.concatenate([us_val, mx_val])

train_output = np.ones((us_train.shape[0], 1))
train_output = np.concatenate([train_output, np.zeros((mx_train.shape[0], 1))])
train_output = keras.utils.to_categorical(train_output)
val_output = np.ones((us_val.shape[0], 1))
val_output = np.concatenate([val_output, np.zeros((mx_val.shape[0], 1))])
val_output = keras.utils.to_categorical(val_output)


print(train_input.shape, "\n", val_input.shape, "\n", train_output.shape, "\n", val_output.shape)
print(train_input.dtype, "\n", val_input.dtype, "\n", train_output.dtype, "\n", val_output.dtype)

(3917, 100, 26) 
 (397, 100, 26) 
 (3917, 2) 
 (397, 2)
float64 
 float64 
 float32 
 float32


### Define Model

In [16]:
out_index = 2
model = keras.Sequential()
model.add(LSTM(25, input_shape=(w_length, train_input.shape[2]), dropout = .5, recurrent_dropout = .5))
model.add(Dense(n_countries, activation= "softmax"))
model.compile(loss = "categorical_crossentropy", optimizer= "adam", metrics=["acc"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 25)                5200      
_________________________________________________________________
dense (Dense)                (None, 2)                 52        
Total params: 5,252
Trainable params: 5,252
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
type(train_input)
train_input.dtype

dtype('float64')

In [18]:
model.fit(train_input, train_output,
          epochs = 50, shuffle = True,
          validation_data = (val_input, val_output),
          batch_size = 128,
         use_multiprocessing = True)

Train on 3917 samples, validate on 397 samples
Epoch 1/50
3917/3917 [==============================] - 3s 745us/sample - loss: 0.8168 - acc: 0.4542 - val_loss: 0.7626 - val_acc: 0.4861
Epoch 2/50
3917/3917 [==============================] - 1s 377us/sample - loss: 0.7288 - acc: 0.5198 - val_loss: 0.6976 - val_acc: 0.5038
Epoch 3/50
3917/3917 [==============================] - 1s 366us/sample - loss: 0.7041 - acc: 0.5484 - val_loss: 0.6676 - val_acc: 0.6322
Epoch 4/50
3917/3917 [==============================] - 1s 368us/sample - loss: 0.6785 - acc: 0.5788 - val_loss: 0.6530 - val_acc: 0.6448
Epoch 5/50
3917/3917 [==============================] - 1s 373us/sample - loss: 0.6715 - acc: 0.5977 - val_loss: 0.6444 - val_acc: 0.6322
Epoch 6/50
3917/3917 [==============================] - 1s 378us/sample - loss: 0.6677 - acc: 0.6035 - val_loss: 0.6374 - val_acc: 0.6448
Epoch 7/50
3917/3917 [==============================] - 1s 368us/sample - loss: 0.6661 - acc: 0.6017 - val_loss: 0.6265 - val